<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Load Data</a></span></li></ul></li><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Train Test Split</a></span></li><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Logistic Regression</a></span><ul class="toc-item"><li><span><a href="#Upsampling-for-imbalanced-target-variable" data-toc-modified-id="Upsampling-for-imbalanced-target-variable-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Upsampling for imbalanced target variable</a></span></li></ul></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load
import warnings
%matplotlib inline
sns.set(style = "whitegrid")
sns.set_palette("icefire")
plt.rcParams['figure.figsize'] = (9, 6)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings(action="ignore")

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold, GridSearchCV
import sklearn.model_selection as ms
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import resample

## Load Data

In [34]:
providers = pd.read_csv('providers_final.csv', index_col = 'Provider')
providers.drop('Unnamed: 0',1,inplace=True)
providers = providers.fillna(0)

In [35]:
providers.shape

(5410, 43)

In [16]:
providers.head()

,Patient_Attphy_Ratio,Patient_Operphy_Ratio,Patient_Otherphy_Ratio,Claim_Patient_Ratio,Claim_AttPhy_Ratio,IsOutpatient_Perc,BeneID_Nunique_IP,State_Nunique_IP,BeneID_Nunique_OP,State_Nunique_OP,AllPhy_mean_IP,NoPhy_mean_IP,ClaimDuration_mean_IP,InscClaimAmtReimbursed_mean_IP,AdmisDuration_mean_IP,AgeAtClm_mean_IP,DeductibleAmtPaid_mean_IP,Chronic_Sum_mean_IP,InsCovRatio_mean_IP,RevPerDay_mean_IP,AllPhy_mean_OP,NoPhy_mean_OP,ClaimDuration_mean_OP,InscClaimAmtReimbursed_mean_OP,AdmisDuration_mean_OP,AgeAtClm_mean_OP,DeductibleAmtPaid_mean_OP,Chronic_Sum_mean_OP,InsCovRatio_mean_OP,RevPerDay_mean_OP,Att_Phy_Mult_Prec,Oper_Phy_Mult_Prec,Other_Phy_Mult_Prec,IP_Multiple_Hospital_Prec,OP_Multiple_Hospital_Prec,Bene_Receive_Both_IO_Perc,Provider_Serve_BothIO,IP_Dup_Perc,OP_Dup_Perc,OP_No_Diag_Perc,IP_No_Proc_Perc,In_Top5_State_Perc,PotentialFraud
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,10,19,16,1.041667,1.785714,0.800000,5.0,1.0,19.0,1.0,0.400000,0.0,6.000000,19400.000000,5.000000,77.800000,1068.0,6.000000,0.878279,3050.378571,0.150000,0.000000,1.550000,382.000000,0.0,78.250000,0.000000,5.450000,1.000000,160.545455,0.240000,0.040000,0.000000,0.160000,0.920000,0.400000,1.0,0.000000,0.550000,0.000000,0.600000,0.0,0
PRV51003,73,110,95,1.128205,3.000000,0.530303,53.0,3.0,66.0,3.0,0.645161,0.0,6.161290,9241.935484,5.161290,70.370968,1068.0,4.919355,0.821059,1828.071976,0.071429,0.000000,3.357143,466.714286,0.0,68.900000,1.000000,4.214286,0.994032,171.554690,0.007576,0.000000,0.000000,0.045455,0.818182,0.500000,1.0,0.016129,0.357143,0.000000,0.370968,0.0,1
PRV51004,100,119,112,1.079710,3.921053,1.000000,0.0,0.0,138.0,9.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.181208,0.000000,2.429530,350.134228,0.0,71.778523,2.080537,4.342282,0.978485,130.318380,0.167785,0.013423,0.000000,0.060403,0.899329,0.208054,0.0,0.000000,0.461538,0.040268,0.000000,0.0,0
PRV51005,489,489,491,2.353535,194.166667,1.000000,0.0,0.0,495.0,4.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.190558,0.001717,2.088412,241.124464,0.0,70.082403,3.175966,4.335622,0.980747,100.999242,0.000000,0.000000,0.000000,0.048069,0.731330,0.248069,0.0,0.000000,0.452257,0.011159,0.000000,0.0,1
PRV51007,48,53,51,1.241379,7.200000,0.958333,3.0,1.0,56.0,2.0,0.333333,0.0,6.333333,6333.333333,5.333333,78.000000,1068.0,5.666667,0.829955,1070.380952,0.159420,0.000000,1.768116,213.188406,0.0,68.391304,0.869565,4.101449,0.992157,100.455505,0.597222,0.083333,0.027778,0.069444,0.902778,0.277778,1.0,0.000000,0.420290,0.000000,0.666667,0.0,0


# Logistic Regression

In [44]:
logistic = LogisticRegression(solver="liblinear")
skf = StratifiedKFold(n_splits=5,random_state=0, shuffle= True)

## Upsampling for imbalanced target variable

In [63]:
# Separate majority and minority classes
providers_majority = providers[providers['PotentialFraud'] == 0]
providers_minority = providers[providers['PotentialFraud'] == 1]
 
# Upsample minority class
providers_minority_upsampled = resample(providers_minority,
                                        replace = True,     # sample with replacement
                                        n_samples = 4904,    # to match majority class
                                        random_state = 0) # reproducible results
 
# Combine majority class with upsampled minority class
providers_upsampled = pd.concat([providers_majority, providers_minority_upsampled])
 
# Display new class counts
providers_upsampled['PotentialFraud'].value_counts()

1    4904
0    4904
Name: PotentialFraud, dtype: int64

In [65]:
# Separate input features (X) and target variable (y)
y = providers_upsampled['PotentialFraud']
X = providers_upsampled.drop('PotentialFraud', axis=1)

In [68]:
# Train model
logistic.fit(X, y)
pred_y = logistic.predict(X)
accuracy_score(y, pred_y)

0.8768352365415987

In [69]:
scores = ms.cross_val_score(estimator=logistic, X=X, y=y, cv=skf)
np.mean(scores)

0.864089500613648

In [70]:
confusion_matrix(y, pred_y)

array([[4239,  665],
       [ 543, 4361]])